In [ ]:
# Import data packages
import pandas as pd
from pathlib import Path
import warnings

# Import database packages
from sqlalchemy import create_engine, select
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session, declarative_base



In [ ]:
#Global configs
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [ ]:
#Open data file into csv
file_path = Path("./data/zillow_listed_prices_052924.csv")

housing_data = pd.read_csv(file_path)
housing_data.head()

In [ ]:
housing_data.columns

In [ ]:
corr_data = housing_data[['area', 'baths', 'beds', 'latLong/latitude', 'latLong/longitude']].corrwith(housing_data['unformattedPrice'])
corr_data

In [ ]:
housing_data.columns

In [ ]:
# Drop unnecessary columns
model_df = housing_data[['id', 'area', 'baths', 'beds', 'latLong/latitude', 'latLong/longitude', 'unformattedPrice']]
model_df.head()

In [ ]:
# Create df for reference data
ref_df = housing_data[['id', 'price', 'addressCity', 'addressZipcode', 'area', 'baths',
        'beds', 'latLong/latitude', 'latLong/longitude',  'statusText']]
ref_df.head()

In [ ]:
# Start model data cleaning

#rename columns
model_df = model_df.rename(columns={'id': 'id', 'area': 'sqft', 'latLong/latitude': 'latitude', 
                                    'latLong/longitude': 'longitude', 'unformattedPrice': 'sale_price'})

#Confirm final data
model_df.head()

In [ ]:
#Start reference data cleansing
ref_df = ref_df.rename(columns={'addressCity': 'city', 'addressZipcode': 'zipcode', 'area': 'sqft', 
                        'latLong/latitude': 'latitude', 'latLong/longitude': 'longitude',  'statusText': 'type'})

#cleanup house type
ref_df['type'] = ref_df.apply(lambda row: row['type'].split(" ")[0], axis=1)

ref_df.head()

In [ ]:
#Get model columns for database
model_df.columns

In [ ]:
#Get reference columns for database
ref_df.columns

In [ ]:
# Begin data classes and storage
Base = declarative_base()

# Create a connection to a SQLite database
engine = create_engine(f"sqlite:///housing_model.db")

In [ ]:
#Define model and reference tables

class ModelData(Base):
    __tablename__ = 'model_data'
    id = Column(Integer, primary_key=True)
    beds = Column(Integer)
    baths = Column(Integer)
    sqft = Column(Integer)
    latitude = Column(Integer)
    longitude = Column(Integer)
    price = Column(Integer)

class RefData(Base):
    __tablename__ = 'reference_data'
    id = Column(Integer, primary_key=True)
    price = Column(String)
    city = Column(String)
    zipcode = Column(String)
    sqft = Column(Integer)
    baths = Column(Integer)
    beds = Column(Integer)
    latitude = Column(Float)
    longitude = Column(Float)
    type = Column(String)

In [ ]:
# Create the travel_destinations table within the database
Base.metadata.create_all(engine)

In [ ]:
# Loop thru dataframes and write to tables

with Session(engine) as session:
    #Loop thru model data
    for index, row in model_df.iterrows():
        model = ModelData()
        model.id = row['id']
        model.beds = row['beds']
        model.baths = row['baths']
        model.sqft = row['sqft']
        model.latitude = row['latitude']
        model.longitude = row['longitude']
        model.price = row['sale_price']

        session.add(model)

    #commit after adding all rows for each model
    session.commit()

    #Loop thru reference data
    for index, row in ref_df.iterrows():
        ref = RefData()
        ref.id = row['id']
        ref.price = row['price']
        ref.city = row['city']
        ref.zipcode = row['zipcode']
        ref.sqft = row['sqft']
        ref.baths = row['baths']
        ref.beds = row['beds']
        ref.latitude = row['latitude']
        ref.longitude = row['longitude']
        ref.type = row['type']

        session.add(ref)

    #commit after adding all rows for each model
    session.commit()
    

In [ ]:
#Validate data in database
#Model data
stmt = select(ModelData)
results = session.execute(stmt).mappings().all()
for row in results:
    print(f'ID: {row.ModelData.id} - Price: {row.ModelData.price}')

In [ ]:
#Reference data
stmt = select(RefData)
results = session.execute(stmt).mappings().all()
for row in results:
    print(f'ID: {row.RefData.id} - Price: {row.RefData.price}')

In [ ]:
session.close()